In [1]:
import os
import sqlite3
import pandas as pd
import csv

In [2]:
try:
    os.remove('airline_v2.db')
except OSError:
    pass


In [3]:
conn = sqlite3.connect('airline_v2.db')

In [4]:
airports = pd.read_csv("../../SIM/ST2195/dataverse_files/airports.csv")
carriers = pd.read_csv("../../SIM/ST2195/dataverse_files/carriers.csv")
planes = pd.read_csv("../../SIM/ST2195/dataverse_files/plane-data.csv")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)


In [5]:
c = conn.cursor()

In [6]:
c.execute('''
CREATE TABLE ontime(
  Year int,
  Month int,
  DayofMonth int,
  DayofWeek int,
  DepTime int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')
conn.commit()

In [7]:
c.execute('''
SELECT name
FROM sqlite_master
WHERE type='table'
''').fetchall()

[('airports',), ('carriers',), ('planes',), ('ontime',)]

In [8]:
for year in range(2000, 2006):
    filename = "../../SIM/ST2195/dataverse_files/"+str(year)+".csv.bz2"
    print('Processing: ', year)
    ontime = pd.read_csv(filename, dtype={'Cancelled': int},encoding='latin-1')
    ontime.to_sql('ontime', con = conn, if_exists= 'append', index = False)
    
conn.commit()

Processing:  2000
Processing:  2001
Processing:  2002
Processing:  2003


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processing:  2004
Processing:  2005


# Query 1 - Plane model with highest departure delay

In [22]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime ON planes.tailnum=ontime.Tailnum
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

x = c.fetchall()
print(x[0][0], "has the lowest associated average departure delay at", round(x[0][1],3))
#print(c.fetchone()[0], "has the lowest associated average departure delay")

737-2Y5 has the lowest associated average departure delay at 7.022


In [23]:
x[:5]

[('737-2Y5', 7.022026431718062),
 ('737-282', 8.433566433566433),
 ('737-230', 10.458646616541353),
 ('737-282C', 11.765765765765765),
 ('767-324', 15.788051209103841)]

In [24]:
df = pd.DataFrame(x)
df

,0,1
0,737-2Y5,7.022026
1,737-282,8.433566
2,737-230,10.458647
3,737-282C,11.765766
4,767-324,15.788051
...,...,...
141,ATR 72-212,38.267365
142,ATR-72-212,38.699195
143,717-200,38.737105
144,ERJ 190-100 IGW,41.723958


In [25]:
df.to_csv("q1.csv", sep=',',index=False)

# Query 2 - City with highest number of inbound fligts

In [18]:
c.execute('''
SELECT airports.city AS city, COUNT() AS total
FROM airports JOIN ontime ON ontime.dest=airports.iata
WHERE ontime.Cancelled = 0 
GROUP BY airports.city
ORDER BY total DESC
''')

s = c.fetchall()
print(s[0][0], "has the highest number of inbound fligts (excluding cancelled flights) at", s[0][1])
#print(c.fetchone()[0], "has the highest number of inbound fligts (excluding cancelled flights)")

Chicago has the highest number of inbound fligts (excluding cancelled flights) at 2388365


In [19]:
s[:5]

[('Chicago', 2388365),
 ('Atlanta', 1912370),
 ('Dallas-Fort Worth', 1781833),
 ('Houston', 1304407),
 ('Los Angeles', 1279514)]

In [20]:
df2 = pd.DataFrame(s)
df2

,0,1
0,Chicago,2388365
1,Atlanta,1912370
2,Dallas-Fort Worth,1781833
3,Houston,1304407
4,Los Angeles,1279514
...,...,...
282,Provo,3
283,Cheyenne,2
284,Scottsbluff,1
285,Roswell,1


In [21]:
df2.to_csv("q2.csv", sep=',',index=False)

# Query 3 - Carrier with the highest number of cancelled flights

In [26]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT() AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier=carriers.code
WHERE ontime.Cancelled = 1 
GROUP BY carriers.Description
ORDER BY total DESC
''')

y = c.fetchall()
print(y[0][0], "has the highest number of cancelled flights at", y[0][1])
#print(c.fetchone()[0], "has the highest number of cancelled flights")

Delta Air Lines Inc. has the highest number of cancelled flights at 107851


In [27]:
y[:5]

[('Delta Air Lines Inc.', 107851),
 ('American Airlines Inc.', 105762),
 ('United Air Lines Inc.', 102066),
 ('American Eagle Airlines Inc.', 96365),
 ('US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.)',
  85866)]

In [28]:
df3 = pd.DataFrame(y)
df3

,0,1
0,Delta Air Lines Inc.,107851
1,American Airlines Inc.,105762
2,United Air Lines Inc.,102066
3,American Eagle Airlines Inc.,96365
4,US Airways Inc. (Merged with America West 9/05...,85866
5,Southwest Airlines Co.,65549
6,Northwest Airlines Inc.,59210
7,Atlantic Southeast Airlines,27947
8,Alaska Airlines Inc.,26667
9,Continental Air Lines Inc.,26565


In [29]:
df3.to_csv("q3.csv", sep=',',index=False)

# Query 4 - Carrier with the highest ratio of cancelled flights to total flights

In [ ]:
c.execute('''
SELECT 
q1.carrier as carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio 
FROM
(
  SELECT carriers.Description AS carrier, count() AS numerator
  FROM carriers JOIN ontime on ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, count() AS denominator
  FROM carriers JOIN ontime on ontime.UniqueCarrier = carriers.Code
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')

d = c.fetchall()
print(d[0][0], "has the highest number of cancelled flights, relative to their number of total flights, at", round(d[0][1],3))

#print(c.fetchone()[0], "has the highest number of cancelled flights, relative to their number of total flights")

In [ ]:
d[:5]

In [ ]:
df4 = pd.DataFrame(d)
df4

In [ ]:
df4.to_csv("q4.csv", sep=',',index=False)